# Sentinel-1

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio

import asf_search as asf
from hyp3_sdk import HyP3

from vegmapper import s1
from vegmapper import pathurl

## User Inputs

In [ ]:
# Site name
sitename = 'malaysia'

# Project directory (S3 path or local path)
# proj_dir = 's3://cropsamz/tests/malaysia'
proj_dir = pathurl.ProjDir(f'{sitename}')

# AOI file
aoifile = f'{sitename}/peninsular_malaysia.geojson'

# Start and end dates of interest
start_date = '2021-01-01'
end_date = '2021-02-01'

In [ ]:
# Plot AOI
gdf_aoi = gpd.read_file(aoifile)
gdf_aoi.plot(figsize=(10, 10))

## Granule Search

ASF DAAC uses *granules* and *scenes* interchangeably to refer to a Sentinel-1 product temporally and geographically, whereas *frames* are used to refer to the geolocation only for a Sentinel-1 product. The naming convention for a Sentinel-1 granule can be found [here](https://asf.alaska.edu/data-sets/sar-data-sets/sentinel-1/sentinel-1-data-and-imagery/). Each *frame* can be uniquely identified by a pair of *path* and *frame* numbers. In this section, we will search for Sentinel-1 granules that intersect with AOI and were acquired between the start and end dates.

### `s1.search_granules`

```
s1.search_granules(sitename, aoifile, start_date, end_date, skim=True, **search_opts)
```

Paremeters:

|Paremeters||Description||Required||Default|
|----||----||----||----|
|`sitename`||Site name||Yes|||
|`aoifile`||AOI file in vector-based spatial data format (shapefile, GeoJSON, ...)||Yes|||
|`start_date`||Start date (YYYY-MM-DD)||Yes|||
|`end_date`||End date (YYYY-MM-DD)||Yes|||
|`skim`||Skim the search results so only the frames that just cover the AOI are retained||No||True|
|`search_opts`||Search options for ASF Python module (asf_search). See [here](https://docs.asf.alaska.edu/asf_search/searching/).||No||True|

Returns:

|Returns||Description|
|----||----|
|`gdf_granules`||A GeoDataFrame containing all searched granules along with their detailed properties|
|`gdf_frames`||A GeoDataFrame of `gdf_granules` grouped by frames.|

In [ ]:
# Here we search for Sentinel-1 Ground Range Detected (GRD) High Resolution (HD) products acquired with Interferometric Wide (IW) beam mode and both VV and VH polarizations.
search_opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'processingLevel': asf.PRODUCT_TYPE.GRD_HD,
    'beamMode': asf.BEAMMODE.IW,
    'polarization': asf.POLARIZATION.VV_VH,
}
gdf_granules, gdf_frames = s1.search_granules(sitename, aoifile, start_date, end_date, skim=True, **search_opts)

In [ ]:
gdf_granules.info()

In [ ]:
gdf_frames

In [ ]:
# Plot search results
ax = gdf_aoi.plot(figsize=(10, 10))
gdf_granules.boundary.plot(ax=ax, color='red')

## Radiometric Terrain Correction (RTC)

For the initial processing of the Sentinel-1 granules, we make use of ASF's HyP3 API. Information about the specifics of this processing can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/). `s1_submit_hyp3_jobs.py` will submit the granules chosen in the previous step to the HyP3 API for processing. The processed granules will be saved in the following directory structure, as .zip files.

### `s1.submit_hyp3_jobs`

```
s1.submit_hyp3_jobs(granules, hyp3=None, proj_dir=None, **rtc_opts)
```

|Parameters|Description|Required|Default|
|----|----|----|----|
|`granules`|GeoJSON file or gdf_granules|Yes||
|`hyp3`|HyP3 API.|No|None|
|`proj_dir`|Project directory|No|None|
|`rtc_opts`|RTC processing options|No|None|

In [ ]:
hyp3 = HyP3(prompt=True)

In [ ]:
# RTC processing options (https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job)
rtc_opts = {
    'dem_matching': True,
    'dem_name': 'copernicus',
    'include_dem': False,
    'include_inc_map': True,
    'include_rgb': False,
    'include_scattering_area': False,
    'radiometry': 'gamma0',
    'resolution': 30,
    'scale': 'power',
    'speckle_filter': False,
}
batch = s1.submit_rtc_jobs(gdf_granules, proj_dir, hyp3, resubmit=False, **rtc_opts)

In [ ]:
batch = hyp3.find_jobs().filter_jobs(include_expired=False)
s1.download_files(batch, download_dir='hyp3_downloads')

In [ ]:
s1.copy_files(proj_dir, download_dir='hyp3_downloads')

## Temporal Statistics

### `s1.s1_proc`

The final processing step involves calculating the temporal mean for the Sentinel-1 granules and removing left/right (cross-track) edge pixels where border noise is prominent. `s1_proc.py` handles this final processing step. Two helper scripts in the Sentinel directory are used: `s1_build_vrt.py` and `calc_vrt_stats.py` for the calculating the temporal mean, and `remove_edges.py` for removing edge pixels. The results will be stored in the following directory structure, as .tif and .vrt files:
```
        proj_dir
           └──sentinel_1
               └──<year>
                   └──<path_frame>
```

##### Usage

```
s1_proc(proj_dir, year, m1, m2, path_frame=None)
```

|Arguments||Description||Required||Default|
|----||----||----||----|
|`path_frame`||If specified, only granules matching `path_frame` will be processed.||No||all path_frames under srcpath/year will be processed|
|`m1`||If specified, only granules with acquisition month >= `m1` will be included in VRT||No||None|
|`m2`||If specified, only granules with acquisition month <= `m2` will be included in VRT||No||None|
|`proj_dir`||Your project directory (see [section 3](#3.-Set-up-project-directory)).||Yes||None|
|`year`||year of Sentinel-1 dataset||Yes||None|

##### Notes

* The processing will be slow if `proj_dir` is on AWS S3 or GCS because it requires heavy network I/O between the cloud and your local machine. If `proj_dir` is on AWS S3, it is strongly recommended that you run the processing on AWS EC2.

Perform the final processing step for our Sentinel-1 granules by running the cell below:

In [ ]:
df = pd.read_csv('/home/jovyan/notebooks/VegMapper/projects/malaysia/Sentinel-1/rtc_products.csv')
df.info()

In [ ]:
s1.build_vrt(proj_dir, start_date, end_date, 'VV')

In [ ]:
!calc_vrt_stats.py /home/jovyan/notebooks/VegMapper/projects/malaysia/Sentinel-1/70_1187/2021-01-01_2021-02-01/VV.vrt mean

In [ ]:
with rasterio.open('/home/jovyan/notebooks/VegMapper/projects/malaysia/Sentinel-1/70_1187/2021-01-01_2021-02-01/VV_mean.tif') as dset:
    VV = dset.read(1)
VV[VV == 0] = np.nan
plt.imshow(VV, vmin=0, vmax=0.5)

In [ ]:
plt.imshow(VV[1750:2000, 0:200], vmin=0, vmax=0.5)